# AWSDet Sagemaker Tutorial

This tutorial walks through the entire process of training an object detection model on Sagemaker. The tutorial does not go into the details of the model itself. Rather, we focus only on setting up and training the model. For a detailed explanation of individual models, please look at resources mentioned in the README.md for that model.

We assume that you are running this notebook on a SageMaker Notebook instance. For instructions on how to setup a SageMaker Notebook instance for this tutorial, please see [README.md](./README.md)

The following are the major components that are required,
- Data download and preparation
- Docker container
- Hyperparameter settings
- Launching a model
- Monitoring

### Data download and preparation
The `prep_data_s3.sh` shell script contains everything to download the COCO 2017 dataset, as well as resnet weights pretrained on imagenet data. These are the same weights included in the Keras package. We download them manually for stability, because they are sometimes retrained between versions of Keras. The script will download everything to the Sagemaker notebook instance, assemble everything in the file structure the training script expects to see, archives it, and saves it to your S3 bucket.

In [1]:
%%bash
cd ~/SageMaker
echo "Current dir: $PWD"
# IMPORTANT if deep-learning-models directory already exists then backup (optional) and remove it so that git clone can succeed
git clone https://github.com/aws-samples/deep-learning-models
cd deep-learning-models/models/vision/detection/
# IMPORTANT replace argument to the script with name of your s3 bucket not including s3://
scripts/prep_data_s3.sh mzanur-sagemaker

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Docker container

While the data preparation script mentioned above is running, you can prepare a Docker image for SageMaker. When you launch a Sagemaker training job, each instance in that job will download a Docker image from Elastic Container Registry (ECR), setup a series of environment variables so it knows your hyperparameters and where to find the data, and launches your training script. Most of the time, you can use Sagemaker's built in containers, but for customized models you could have to create your own.

Under `deep-learning-models/models/vision/detection/docker` you'll find the docker file `Dockerfile.sagemaker` that creates the image that we use for training. A detailed description of that file, and how to create your own, is included in  [README.md](../docker/README.md). Below are the basic commands to build this image on your SageMaker notebook instance and upload it to ECR.

In [2]:
%%bash
cd ~/SageMaker/deep-learning-models/models/vision/detection/
# IMPORTANT replace argument to the script with name of the ECR repository that you created
scripts/build_docker_sagemaker.sh mzanur-awsdet-ecr

Sending build context to Docker daemon  528.9kB
Step 1/43 : FROM nvidia/cuda:10.1-base-ubuntu18.04
 ---> 3b55548ae91f
Step 2/43 : LABEL maintainer="Amazon AI"
 ---> Using cache
 ---> c56dde051dc6
Step 3/43 : ENV DEBIAN_FRONTEND noninteractive
 ---> Using cache
 ---> 5dffef1a4cc4
Step 4/43 : ENV DEBCONF_NONINTERACTIVE_SEEN true
 ---> Using cache
 ---> 7ba83002e360
Step 5/43 : ENV SAGEMAKER_TRAINING_MODULE sagemaker_tensorflow_container.training:main
 ---> Using cache
 ---> 09c2a9f02431
Step 6/43 : ENV PYTHONDONTWRITEBYTECODE=1
 ---> Using cache
 ---> 1b082325b61b
Step 7/43 : ENV PYTHONUNBUFFERED=1
 ---> Using cache
 ---> da3b14f5c907
Step 8/43 : ENV PYTHONIOENCODING=UTF-8
 ---> Using cache
 ---> 297ef8077a08
Step 9/43 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> 28150667cf22
Step 10/43 : ENV LC_ALL=C.UTF-8
 ---> Using cache
 ---> 9febde0a5c70
Step 11/43 : ENV KMP_AFFINITY=granularity=fine,compact,1,0
 ---> Using cache
 ---> ebc26e1cfe08
Step 12/43 : ENV KMP_BLOCKTIME=1
 ---> Using cache
 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



At the end of running the script above it will emit your docker image ID. Please put in that image ID in the configuration for your desired model under `deep-learning-models/models/vision/detection/configs/<model>/SM/<configuration file>` as described in the following section.

### SageMaker and Model configuration

Now we can start setting up the model training process on SageMaker.

Most of the hyperparameters related to the model structure do not require tweaking. Some training related hyperparameters, e.g. learning rates, momentum, weight decay may need changing depending on how many nodes you are using to train, or your dataset, or the batch size per GPU that you decide to choose.

#### Brief explanation of parameters in configuration file

`user_id` can be anything you like, used for keeping track of your training jobs

`s3_bucket` name of your s3 bucket without s3://, e.g. jbsnyder-sagemaker

`docker_image` - ID of the docker image id that was built, e.g. 12345.dkr.ecr.us-east-1.amazonaws.com/name:awsdet

`instance_count` number of instances (nodes) for training.

`instance_type` the type of EC2 instances that will be used for training. For highest performance use `ml.p3dn.24xlarge`. The `ml.p3.16xlarge` and `ml.p3.8xlarge` are slightly lower performance while being more cost effective. The `g4dn.12xlarge` instance type is even lower cost, but much slower than the p3 instances.

`num_workers_per_host` number of training jobs per instance. This should be the number of GPUs on an instance, so should be set to 8 for `ml.p3dn.24xlarge` or `ml.p3.16xlarge` instances, and 4 for `ml.p3.8xlarge` or `g4dn.12xlarge`.

`amp_enabled` refers to whether a model should train using mixed precision. Mixed precision training is supported on p3 and g4 instances.

`warmup_steps` number of steps for which the learning rate will be increased from `learning_rate * warmup_ratio` to `learning_rate`.

`optimizer` this is the configuration for an `tf.keras.optimizers` optimizer. The `learning_rate` specified here is the learning rate adjusted for 8 GPUs after warmup steps. If you have more than 8 GPUs then this rate will be scaled linearly automatically.

`imgs_per_gpu` number of images per GPU.

Other model specific details can be found and tweaked in the relevant code under `deep-learning-models/models/vision/detection/awsdet` directory. This is suggested for advanced users.

### SageMaker Job Launch

In [4]:
%cd ~/SageMaker/deep-learning-models/models/vision/detection/
%run tools/launch_sagemaker_job.py --configuration configs/faster_rcnn/SM/faster_rcnn_r50_fpn_1x_coco.py

/home/ec2-user/SageMaker/deep-learning-models/models/vision/detection
Launched SageMaker job: mzanur-frcnn-24-06-2020-03-48


Go to the Sagemaker homepage and click `Training Jobs`, you should see a new training job with a name that was setup in the configuration file. Click the link to see what your training job is doing.

The training takes about 5-10 minutes to set up, depending on the instance type that was chosen.

The sagemaker job launch script downloads data required to train from S3, then the docker image (that was built) is downloaded from ECR. 

You can monitor CPU/GPU usage and see CloudWatch logs for each instance under the section called `Monitor`.

For a more detailed view into training losses, accuracy, and to visualize predictions, e.g. bounding boxes, Tensorboard can be used. Once training starts, a new set of directories in your S3 bucket are created 

```
[s3_path]/tensorboard/
```

In a terminal on your notebook instance, run

```
conda activate tensorflow_p36
tensorboard --logdir [s3_path]/tensorboard
```